In [ ]:
# Define image paths
real = "real_vs_fake/real-vs-fake/train/real/"
fake = "real_vs_fake/real-vs-fake/train/fake/"


In [ ]:
# Required Libraries
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Sequential, load_model, Model
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dropout, Dense, BatchNormalization, GlobalAveragePooling2D, Add, Multiply, Concatenate, Input, Flatten, Dense
from tensorflow.keras.callbacks import LearningRateScheduler, ModelCheckpoint, EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
import matplotlib.pyplot as plt
import cv2
import os
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score


In [ ]:
# Load and display images
def load_img(path):
    image = cv2.imread(path)
    if image is None:
        raise FileNotFoundError(f"Image not found or unable to load: {path}")
    image = cv2.resize(image, (224, 224))
    return image[..., ::-1]

# Display real images
fig = plt.figure(figsize=(10, 10))
real_path = os.listdir(real)
for i in range(16):
    plt.subplot(4, 4, i + 1)
    plt.imshow(load_img(real + real_path[i]), cmap='gray')
    plt.suptitle("Real faces", fontsize=20)
    plt.axis('off')
plt.show()

# Display fake images
fig = plt.figure(figsize=(10, 10))
fake_path = os.listdir(fake)
for i in range(16):
    plt.subplot(4, 4, i + 1)
    plt.imshow(load_img(fake + fake_path[i]), cmap='gray')
    plt.suptitle("Fake faces", fontsize=20)
    plt.title(fake_path[i][:4])
    plt.axis('off')
plt.show()



In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255)

val_datagen = ImageDataGenerator(rescale=1./255)


train = train_datagen.flow_from_directory('real_vs_fake/real-vs-fake/train/',
                                          target_size=(224, 224),
                                          batch_size=32,
                                          class_mode='sparse',
                                          shuffle=True)

val = val_datagen.flow_from_directory('real_vs_fake/real-vs-fake/valid/',
                                      target_size=(224, 224),
                                      batch_size=32,
                                      class_mode='sparse',
                                      shuffle=True)

Found 100000 images belonging to 2 classes.
Found 20000 images belonging to 2 classes.


In [ ]:
# Clear session
tf.keras.backend.clear_session()

# Input layer
inputs = Input(shape=(224, 224, 3))

# Base model (đóng băng weight)
mnet = tf.keras.applications.MobileNetV2(weights='imagenet', include_top=False, input_tensor=inputs)
mnet.trainable = False

# Thêm các tầng sau
x = mnet.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation="relu")(x)
x = BatchNormalization()(x)
x = Dropout(0.3)(x)
x = Dense(128, activation="relu")(x)
x = Dropout(0.1)(x)
outputs = Dense(2, activation="softmax")(x)

# Model Functional
model = Model(inputs, outputs)

# Compile
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

model.summary()


In [ ]:
# Learning rate scheduler
def scheduler(epoch):
    if epoch <= 2:
        return 0.0001
    elif epoch > 2 and epoch <= 15:
        return 0.00001
    else:
        return 0.000001

lr_callbacks = tf.keras.callbacks.LearningRateScheduler(scheduler)

# Train the model
hist = model.fit(train,
                 epochs=20,
                 callbacks=[lr_callbacks],
                 validation_data=val)

In [ ]:
epochs = 20
train_loss = hist.history['loss']
val_loss = hist.history['val_loss']
train_acc = hist.history['accuracy']
val_acc = hist.history['val_accuracy']
xc = range(epochs)

plt.figure(1,figsize=(7,5))
plt.plot(xc,train_loss)
plt.plot(xc,val_loss)
plt.xlabel('num of Epochs')
plt.ylabel('loss')
plt.title('train_loss vs val_loss')
plt.grid(True)
plt.legend(['train','val'])
#print plt.style.available # use bmh, classic,ggplot for big pictures
plt.style.use(['classic'])

plt.figure(2,figsize=(7,5))
plt.plot(xc,train_acc)
plt.plot(xc,val_acc)
plt.xlabel('num of Epochs')
plt.ylabel('accuracy')
plt.title('train_acc vs val_acc')
plt.grid(True)
plt.legend(['train','val'],loc=4)
#print plt.style.available # use bmh, classic,ggplot for big pictures
plt.style.use(['classic'])

In [ ]:
# Sau khi huấn luyện xong, lưu mô hình thành checkpoint
model.save('real_vs_fake/checkpoints/model_mobilenetv2_final_20_epochs.keras')

In [ ]:

# Tải mô hình từ checkpoint sau 20 epochs
model = load_model('real_vs_fake/checkpoints/model_mobilenetv2_final_20_epochs.keras')


In [ ]:
# Đường dẫn tới thư mục test
test_path = "real_vs_fake/real-vs-fake/test/"

# ImageDataGenerator cho tập test, chỉ cần rescale
test_datagen = ImageDataGenerator(rescale=1./255)

# Load dữ liệu từ thư mục test
test = test_datagen.flow_from_directory(test_path,
                                        target_size=(224, 224),
                                        batch_size=32,
                                        class_mode='sparse',
                                        shuffle=False)  # Không xáo trộn để giữ đúng thứ tự khi dự đoán


Found 20000 images belonging to 2 classes.


In [ ]:

# Dự đoán kết quả cho tập test
predictions = model.predict(test)
# Lấy tất cả dữ liệu test
X_test, y_true = [], []
for i in range(len(test)):
    X_batch, y_batch = test[i]
    X_test.extend(X_batch)
    y_true.extend(y_batch)

X_test = np.array(X_test)
y_true = np.array(y_true)

# Chọn ngẫu nhiên 500 hình (250 mỗi lớp)
np.random.seed(42)  # Để kết quả có thể tái lập
class_0_idx = np.where(y_true == 0)[0]  # Fake
class_1_idx = np.where(y_true == 1)[0]  # Real

selected_0 = np.random.choice(class_0_idx, 250, replace=False)
selected_1 = np.random.choice(class_1_idx, 250, replace=False)
selected_idx = np.concatenate([selected_0, selected_1])

X_test_sample = X_test[selected_idx]
y_true_sample = y_true[selected_idx]

# Dự đoán kết quả cho tập test sample
predictions = model.predict([X_test_sample])
y_pred = np.argmax(predictions, axis=1)

# In báo cáo phân loại
print("=== 📄 Classification Report (500 samples) ===")
print(classification_report(y_true_sample, y_pred, target_names=["Fake", "Real"]))

# In confusion matrix
cm = confusion_matrix(y_true_sample, y_pred)
print("\n=== 🔍 Confusion Matrix ===")
print(cm)

# Tính các chỉ số tổng thể
if len(test.class_indices.keys()) == 2:  # Nếu là binary classification
    acc = accuracy_score(y_true_sample, y_pred)
    prec = precision_score(y_true_sample, y_pred)
    rec = recall_score(y_true_sample, y_pred)
    f1 = f1_score(y_true_sample, y_pred)
    tn, fp, fn, tp = cm.ravel()

    print(f"\n✅ Overall Accuracy: {acc:.4f}")
    print(f"✅ Overall Precision: {prec:.4f}")
    print(f"✅ Overall Recall: {rec:.4f}")
    print(f"✅ Overall F1-score: {f1:.4f}")

# In chi tiết cho từng class
print("\n=== 📊 Per-Class Statistics ===")
for i, class_name in enumerate(test.class_indices.keys()):
    TP = cm[i,i]
    FP = cm[:,i].sum() - TP
    FN = cm[i,:].sum() - TP
    TN = cm.sum() - TP - FP - FN

    # Tính các chỉ số cho từng class
    accuracy = (TP + TN) / (TP + TN + FP + FN) if (TP + TN + FP + FN) != 0 else 0
    precision = TP / (TP + FP) if (TP + FP) != 0 else 0
    recall = TP / (TP + FN) if (TP + FN) != 0 else 0
    f1_score_class = 2 * (precision * recall) / (precision + recall) if (precision + recall) != 0 else 0

    print(f"\n📌 Class: {class_name}")
    print(f"🟩 True Positive (TP): {TP}")
    print(f"🟥 False Positive (FP): {FP}")
    print(f"🟨 False Negative (FN): {FN}")
    print(f"🟦 True Negative (TN): {TN}")
    print(f"\n📊 Class Metrics:")
    print(f"✅ Accuracy: {accuracy:.4f}")
    print(f"✅ Precision: {precision:.4f}")
    print(f"✅ Recall: {recall:.4f}")
    print(f"✅ F1-score: {f1_score_class:.4f}")

# Vẽ confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
           xticklabels=test.class_indices.keys(),
           yticklabels=test.class_indices.keys())
plt.title('Confusion Matrix (500 samples)')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()


In [ ]:
def make_gradcam_heatmap(img_array, model, last_conv_layer_name="out_relu"):
    # Truy xuất layer conv cuối cùng trong backbone (MobileNetV2)
    last_conv_layer = model.get_layer(last_conv_layer_name)

    # Tạo model phụ để lấy output của lớp conv cuối và output cuối cùng
    grad_model = tf.keras.models.Model(
        inputs=model.input,
        outputs=[last_conv_layer.output, model.output]
    )

    # Tính gradient của đầu ra (class được dự đoán) với output của conv layer
    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_array)
        pred_index = tf.argmax(predictions[0])
        class_channel = predictions[:, pred_index]

    # Gradient của output theo conv_outputs
    grads = tape.gradient(class_channel, conv_outputs)

    # Lấy trung bình gradient trên không gian HxW
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    # Nhân conv_outputs với trọng số từ pooled_grads
    conv_outputs = conv_outputs[0]  # Bỏ batch dim
    heatmap = conv_outputs @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)

    # Chuẩn hóa heatmap về [0, 1]
    heatmap = tf.maximum(heatmap, 0) / (tf.reduce_max(heatmap) + 1e-8)

    return heatmap.numpy()

plt.figure(figsize=(20, 40))

# Get the minimum length among test.filenames, y_true, and y_pred
min_length = min(len(test.filenames), len(y_true), len(y_pred))

for i in range(16):
    # Generate random index within the valid range
    idx = np.random.randint(0, min_length)

    # Load image
    img_path = test.filepaths[idx]
    img = cv2.imread(img_path)
    img = cv2.resize(img, (224, 224))
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Preprocess
    img_array = img_rgb.astype("float32") / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    try:
        # Predict to ensure model is called
        _ = model.predict(img_array)

        # Create heatmap
        heatmap = make_gradcam_heatmap(img_array, model, "out_relu")

        # Resize and overlay
        heatmap = cv2.resize(heatmap, (224, 224))
        heatmap = np.uint8(255 * heatmap)
        heatmap_color = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
        superimposed_img = cv2.addWeighted(img_rgb, 0.6, heatmap_color, 0.4, 0)

        # Get true and predicted labels
        true_idx = int(y_true[idx]) if isinstance(y_true[idx], np.float32) else y_true[idx]
        pred_idx = int(y_pred[idx]) if isinstance(y_pred[idx], np.float32) else y_pred[idx]

        true_label = list(test.class_indices.keys())[true_idx]
        pred_label = list(test.class_indices.keys())[pred_idx]
        color = 'green' if true_idx == pred_idx else 'red'

        # Original image
        plt.subplot(8, 4, 2*i+1)
        plt.imshow(img_rgb)
        plt.title(f"Original\nTrue: {true_label}\nPred: {pred_label}", color=color)
        plt.axis('off')

        # Grad-CAM image
        plt.subplot(8, 4, 2*i+2)
        plt.imshow(superimposed_img)
        plt.title(f"Grad-CAM\nTrue: {true_label}\nPred: {pred_label}", color=color)
        plt.axis('off')

    except Exception as e:
        print(f"Error processing image {idx}: {str(e)}")
        continue

plt.tight_layout()
plt.show()


In [ ]:
# Clear session
tf.keras.backend.clear_session()

# Input layer
inputs = Input(shape=(224, 224, 3))

# Base model (đóng băng weight)
vgg16 = tf.keras.applications.VGG16(weights='imagenet', include_top=False, input_tensor=inputs)
vgg16.trainable = False

# Thêm các tầng sau
x = vgg16.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation="relu")(x)
x = BatchNormalization()(x)
x = Dropout(0.3)(x)
x = Dense(128, activation="relu")(x)
x = Dropout(0.1)(x)
outputs = Dense(2, activation="softmax")(x)

# Model Functional
model = Model(inputs, outputs)

# Compile
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 224, 224, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 224, 224, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 112, 112, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 112, 112, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 56, 56, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 28, 28, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 28, 28, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 14, 14, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 7, 7, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 512)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │       262,656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 512)            │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 128)            │        65,66

 Total params: 15,045,314 (57.39 MB)

 Trainable params: 329,602 (1.26 MB)

 Non-trainable params: 14,715,712 (56.14 MB)

In [ ]:
for layer in model.layers:
    print(layer.name)

In [ ]:
# Learning rate scheduler
def scheduler(epoch):
    if epoch <= 2:
        return 0.0001
    elif epoch > 2 and epoch <= 15:
        return 0.00001
    else:
        return 0.000001

lr_callbacks = tf.keras.callbacks.LearningRateScheduler(scheduler)

# Train the model
hist = model.fit(train,
                 epochs=20,
                 callbacks=[lr_callbacks],
                 validation_data=val)


In [ ]:
epochs = 20
train_loss = hist.history['loss']
val_loss = hist.history['val_loss']
train_acc = hist.history['accuracy']
val_acc = hist.history['val_accuracy']
xc = range(epochs)

plt.figure(1,figsize=(7,5))
plt.plot(xc,train_loss)
plt.plot(xc,val_loss)
plt.xlabel('num of Epochs')
plt.ylabel('loss')
plt.title('train_loss vs val_loss')
plt.grid(True)
plt.legend(['train','val'])
#print plt.style.available # use bmh, classic,ggplot for big pictures
plt.style.use(['classic'])

plt.figure(2,figsize=(7,5))
plt.plot(xc,train_acc)
plt.plot(xc,val_acc)
plt.xlabel('num of Epochs')
plt.ylabel('accuracy')
plt.title('train_acc vs val_acc')
plt.grid(True)
plt.legend(['train','val'],loc=4)
#print plt.style.available # use bmh, classic,ggplot for big pictures
plt.style.use(['classic'])

In [ ]:
# Sau khi huấn luyện xong, lưu mô hình thành checkpoint
model.save('real_vs_fake/checkpoints/model_vgg16_final_20.keras')

In [ ]:
# Tải mô hình từ checkpoint sau 20 epochs
model = load_model('real_vs_fake/checkpoints/model_vgg16_final_20.keras')

I0000 00:00:1743308883.907802    3815 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5803 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3070 Ti, pci bus id: 0000:82:00.0, compute capability: 8.6


In [ ]:
# Đường dẫn tới thư mục test
test_path = "real_vs_fake/real-vs-fake/test/"

# ImageDataGenerator cho tập test, chỉ cần rescale
test_datagen = ImageDataGenerator(rescale=1./255)

# Load dữ liệu từ thư mục test
test = test_datagen.flow_from_directory(test_path,
                                        target_size=(224, 224),
                                        batch_size=32,
                                        class_mode='sparse',
                                        shuffle=False)  # Không xáo trộn để giữ đúng thứ tự khi dự đoán


Found 20000 images belonging to 2 classes.


In [ ]:

# Dự đoán kết quả cho tập test
predictions = model.predict(test)
# Lấy tất cả dữ liệu test
X_test, y_true = [], []
for i in range(len(test)):
    X_batch, y_batch = test[i]
    X_test.extend(X_batch)
    y_true.extend(y_batch)

X_test = np.array(X_test)
y_true = np.array(y_true)

# Chọn ngẫu nhiên 500 hình (250 mỗi lớp)
np.random.seed(42)  # Để kết quả có thể tái lập
class_0_idx = np.where(y_true == 0)[0]  # Fake
class_1_idx = np.where(y_true == 1)[0]  # Real

selected_0 = np.random.choice(class_0_idx, 250, replace=False)
selected_1 = np.random.choice(class_1_idx, 250, replace=False)
selected_idx = np.concatenate([selected_0, selected_1])

X_test_sample = X_test[selected_idx]
y_true_sample = y_true[selected_idx]

# Dự đoán kết quả cho tập test sample
predictions = model.predict([X_test_sample])
y_pred = np.argmax(predictions, axis=1)

# In báo cáo phân loại
print("=== 📄 Classification Report (500 samples) ===")
print(classification_report(y_true_sample, y_pred, target_names=["Fake", "Real"]))

# In confusion matrix
cm = confusion_matrix(y_true_sample, y_pred)
print("\n=== 🔍 Confusion Matrix ===")
print(cm)

# Tính các chỉ số tổng thể
if len(test.class_indices.keys()) == 2:  # Nếu là binary classification
    acc = accuracy_score(y_true_sample, y_pred)
    prec = precision_score(y_true_sample, y_pred)
    rec = recall_score(y_true_sample, y_pred)
    f1 = f1_score(y_true_sample, y_pred)
    tn, fp, fn, tp = cm.ravel()

    print(f"\n✅ Overall Accuracy: {acc:.4f}")
    print(f"✅ Overall Precision: {prec:.4f}")
    print(f"✅ Overall Recall: {rec:.4f}")
    print(f"✅ Overall F1-score: {f1:.4f}")

# In chi tiết cho từng class
print("\n=== 📊 Per-Class Statistics ===")
for i, class_name in enumerate(test.class_indices.keys()):
    TP = cm[i,i]
    FP = cm[:,i].sum() - TP
    FN = cm[i,:].sum() - TP
    TN = cm.sum() - TP - FP - FN

    # Tính các chỉ số cho từng class
    accuracy = (TP + TN) / (TP + TN + FP + FN) if (TP + TN + FP + FN) != 0 else 0
    precision = TP / (TP + FP) if (TP + FP) != 0 else 0
    recall = TP / (TP + FN) if (TP + FN) != 0 else 0
    f1_score_class = 2 * (precision * recall) / (precision + recall) if (precision + recall) != 0 else 0

    print(f"\n📌 Class: {class_name}")
    print(f"🟩 True Positive (TP): {TP}")
    print(f"🟥 False Positive (FP): {FP}")
    print(f"🟨 False Negative (FN): {FN}")
    print(f"🟦 True Negative (TN): {TN}")
    print(f"\n📊 Class Metrics:")
    print(f"✅ Accuracy: {accuracy:.4f}")
    print(f"✅ Precision: {precision:.4f}")
    print(f"✅ Recall: {recall:.4f}")
    print(f"✅ F1-score: {f1_score_class:.4f}")

# Vẽ confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
           xticklabels=test.class_indices.keys(),
           yticklabels=test.class_indices.keys())
plt.title('Confusion Matrix (500 samples)')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()


In [ ]:
def make_gradcam_heatmap(img_array, model, last_conv_layer_name="block5_conv3"):  # Đổi tên layer cho VGG16
    # Truy xuất layer conv cuối cùng trong backbone (VGG16)
    last_conv_layer = model.get_layer(last_conv_layer_name)

    # Tạo model phụ để lấy output của lớp conv cuối và output cuối cùng
    grad_model = tf.keras.models.Model(
        inputs=model.input,
        outputs=[last_conv_layer.output, model.output]
    )

    # Tính gradient của đầu ra (class được dự đoán) với output của conv layer
    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_array)
        pred_index = tf.argmax(predictions[0])
        class_channel = predictions[:, pred_index]

    # Gradient của output theo conv_outputs
    grads = tape.gradient(class_channel, conv_outputs)

    # Lấy trung bình gradient trên không gian HxW
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    # Nhân conv_outputs với trọng số từ pooled_grads
    conv_outputs = conv_outputs[0]  # Bỏ batch dim
    heatmap = conv_outputs @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)

    # Chuẩn hóa heatmap về [0, 1]
    heatmap = tf.maximum(heatmap, 0) / (tf.reduce_max(heatmap) + 1e-8)

    return heatmap.numpy()

plt.figure(figsize=(20, 40))

# Get the minimum length among test.filenames, y_true, and y_pred
min_length = min(len(test.filenames), len(y_true), len(y_pred))

for i in range(16):
    # Generate random index within the valid range
    idx = np.random.randint(0, min_length)

    # Load image
    img_path = test.filepaths[idx]
    img = cv2.imread(img_path)
    img = cv2.resize(img, (224, 224))
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Preprocess
    img_array = img_rgb.astype("float32") / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    try:
        # Predict to ensure model is called
        _ = model.predict(img_array)

        # Create heatmap
        heatmap = make_gradcam_heatmap(img_array, model, "block5_conv3")

        # Resize and overlay
        heatmap = cv2.resize(heatmap, (224, 224))
        heatmap = np.uint8(255 * heatmap)
        heatmap_color = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
        superimposed_img = cv2.addWeighted(img_rgb, 0.6, heatmap_color, 0.4, 0)

        # Get true and predicted labels
        true_idx = int(y_true[idx]) if isinstance(y_true[idx], np.float32) else y_true[idx]
        pred_idx = int(y_pred[idx]) if isinstance(y_pred[idx], np.float32) else y_pred[idx]

        true_label = list(test.class_indices.keys())[true_idx]
        pred_label = list(test.class_indices.keys())[pred_idx]
        color = 'green' if true_idx == pred_idx else 'red'

        # Original image
        plt.subplot(8, 4, 2*i+1)
        plt.imshow(img_rgb)
        plt.title(f"Original\nTrue: {true_label}\nPred: {pred_label}", color=color)
        plt.axis('off')

        # Grad-CAM image
        plt.subplot(8, 4, 2*i+2)
        plt.imshow(superimposed_img)
        plt.title(f"Grad-CAM\nTrue: {true_label}\nPred: {pred_label}", color=color)
        plt.axis('off')

    except Exception as e:
        print(f"Error processing image {idx}: {str(e)}")
        continue

plt.tight_layout()
plt.show()

In [ ]:
# Load MobileNetV2 và VGG16
mobilenet = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
vgg16 = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Đóng băng các lớp của MobileNetV2 và VGG16 để không huấn luyện lại
mobilenet.trainable = False
vgg16.trainable = False

# Lấy đặc trưng từ cả hai mô hình
mobilenet_features = GlobalAveragePooling2D()(mobilenet.output)
vgg16_features = GlobalAveragePooling2D()(vgg16.output)

# Kết hợp các đặc trưng
combined_features = Concatenate()([mobilenet_features, vgg16_features])

# Thêm các lớp fully connected và output
x = Dense(512, activation='relu')(combined_features)
x = Dropout(0.3)(x)
output = Dense(2, activation='softmax')(x)

# Tạo mô hình
model = Model(inputs=[mobilenet.input, vgg16.input], outputs=output)

# Compile mô hình
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Hiển thị tóm tắt mô hình
model.summary()

Model: "functional_17"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1 (Conv2D)      │ (None, 112, 112,  │        864 │ input_layer_1[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn_Conv1            │ (None, 112, 112,  │        128 │ Conv1[0][0]       │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1_relu (ReLU)   │ (None, 112, 112,  │          0 │ bn_Conv1[0][0]    │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        288 │ Conv1_relu[0][0]  │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        128 │ expanded_conv_de… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │          0 │ expanded_conv_de… │
│ (ReLU)              │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │        512 │ expanded_conv_de… │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │         64 │ expanded_conv_pr… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand      │ (None, 112, 112,  │      1,536 │ expanded_conv_pr… │
│ (Conv2D)            │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_BN   │ (None, 112, 112,  │        384 │ block_1_expand[0… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_relu │ (None, 112, 112,  │          0 │ block_1_expand_B… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_pad         │ (None, 113, 113,  │          0 │ block_1_expand_r… │
│ (ZeroPadding2D)     │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise   │ (None, 56, 56,    │        864 │ block_1_pad[0][0] │
│ (DepthwiseConv2D)   │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │        384 │ block_1_depthwis… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │          0 │ block_1_depthwis… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_project     │ (None, 56, 56,    │      2,304 │ block_1_depthwis

 Total params: 17,891,714 (68.25 MB)

 Trainable params: 919,042 (3.51 MB)

 Non-trainable params: 16,972,672 (64.75 MB)

In [ ]:
import tensorflow as tf

def double_data_generator(generator):
    for batch in generator:
        # Ensure it returns a tuple
        yield (tf.convert_to_tensor(batch[0], dtype=tf.float32),
               tf.convert_to_tensor(batch[0], dtype=tf.float32)), \
              tf.convert_to_tensor(batch[1], dtype=tf.float32)
# Chuyển generator thành tf.data.Dataset
train_dataset = tf.data.Dataset.from_generator(
    lambda: double_data_generator(train),
    output_signature=(
        (tf.TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32),
         tf.TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32)),
        tf.TensorSpec(shape=(None,), dtype=tf.float32)
    )
)

val_dataset = tf.data.Dataset.from_generator(
    lambda: double_data_generator(val),
    output_signature=(
        (tf.TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32),
         tf.TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32)),
        tf.TensorSpec(shape=(None,), dtype=tf.float32)
    )
)


In [ ]:
# Learning rate scheduler
def scheduler(epoch):
    if epoch <= 2:
        return 0.0001
    elif epoch > 2 and epoch <= 15:
        return 0.00001
    else:
        return 0.000001

lr_callbacks = tf.keras.callbacks.LearningRateScheduler(scheduler)

history = model.fit(train_dataset,
                    epochs=20,
                    steps_per_epoch=len(train),
                    callbacks=[lr_callbacks],
                    validation_data=val_dataset,
                    validation_steps=len(val))

In [ ]:
# Vẽ đồ thị loss và accuracy
epochs = 20
train_loss = history.history['loss']
val_loss = history.history['val_loss']
train_acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
xc = range(epochs)

plt.figure(1, figsize=(7, 5))
plt.plot(xc, train_loss, label='Train Loss')
plt.plot(xc, val_loss, label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Train Loss vs Validation Loss')
plt.legend()

plt.figure(2, figsize=(7, 5))
plt.plot(xc, train_acc, label='Train Accuracy')
plt.plot(xc, val_acc, label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Train Accuracy vs Validation Accuracy')
plt.legend()
plt.show()

In [ ]:
# Sau khi huấn luyện xong, lưu mô hình thành checkpoint
model.save('real_vs_fake/checkpoints/model_combined_mobilenet_vgg16.keras')

In [ ]:
model = load_model('real_vs_fake/checkpoints/model_combined_mobilenet_vgg16.keras')

In [ ]:
# ================== DATA LOADING ==================
test_path = "real_vs_fake/real-vs-fake/test/"

# ImageDataGenerator for test set
test_datagen = ImageDataGenerator(rescale=1./255)

# Load test data
test = test_datagen.flow_from_directory(
    test_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='sparse',
    shuffle=False
)

print(f"Found {test.samples} images belonging to {len(test.class_indices)} classes.")

# Prepare test data for both inputs
X_test1, X_test2, y_test = [], [], []
for i in range(len(test)):
    X_batch, y_batch = test[i]
    X_test1.extend(X_batch)
    X_test2.extend(X_batch)
    y_test.extend(y_batch)

X_test1 = np.array(X_test1, dtype='float32')
X_test2 = np.array(X_test2, dtype='float32')
y_test = np.array(y_test, dtype='int')

# Balance the test set (500 from each class)
class_0_idx = np.where(y_test == 0)[0][:500]  # Fake
class_1_idx = np.where(y_test == 1)[0][:500]  # Real
balanced_idx = np.concatenate([class_0_idx, class_1_idx])

X_test1 = X_test1[balanced_idx]
X_test2 = X_test2[balanced_idx]
y_test = y_test[balanced_idx]

# ================== GRAD-CAM FUNCTION ==================
def make_gradcam_heatmap(img_array1, img_array2, model, last_conv_layer_name="out_relu"):
    # Create submodel that outputs both conv layer and predictions
    grad_model = Model(
        inputs=model.inputs,
        outputs=[model.get_layer(last_conv_layer_name).output, model.output]
    )

    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model([img_array1, img_array2])
        pred_index = tf.argmax(predictions[0])
        class_channel = predictions[:, pred_index]

    grads = tape.gradient(class_channel, conv_outputs)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
    conv_outputs = conv_outputs[0]
    heatmap = conv_outputs @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    return heatmap.numpy()

# ================== EVALUATION AND VISUALIZATION ==================
# Make predictions
# Make predictions
predictions = model.predict([X_test1, X_test2])
y_pred = np.argmax(predictions, axis=1)

# Classification report
print("\n=== Classification Report ===")
print(classification_report(y_test, y_pred,
                          target_names=test.class_indices.keys(),
                          digits=4))

# Confusion matrix
plt.figure(figsize=(8, 6))
cm = confusion_matrix(y_test, y_pred)

# Calculate TP, TN, FP, FN for each class
# Tính các chỉ số tổng thể (nếu là bài toán binary classification)
if len(test.class_indices.keys()) == 2:  # Nếu là binary classification
    acc = accuracy_score(y_test, y_pred)  # Changed y_true to y_test
    prec = precision_score(y_test, y_pred)  # Changed y_true to y_test
    rec = recall_score(y_test, y_pred)  # Changed y_true to y_test
    f1 = f1_score(y_test, y_pred)  # Changed y_true to y_test
    tn, fp, fn, tp = cm.ravel()

    print(f"\n✅ Overall Accuracy: {acc:.4f}")
    print(f"✅ Overall Precision: {prec:.4f}")
    print(f"✅ Overall Recall: {rec:.4f}")
    print(f"✅ Overall F1-score: {f1:.4f}")

# In chi tiết cho từng class
print("\n=== 📊 Per-Class Statistics ===")
for i, class_name in enumerate(test.class_indices.keys()):
    TP = cm[i,i]
    FP = cm[:,i].sum() - TP
    FN = cm[i,:].sum() - TP
    TN = cm.sum() - TP - FP - FN

    # Tính các chỉ số cho từng class
    accuracy = (TP + TN) / (TP + TN + FP + FN) if (TP + TN + FP + FN) != 0 else 0
    precision = TP / (TP + FP) if (TP + FP) != 0 else 0
    recall = TP / (TP + FN) if (TP + FN) != 0 else 0
    f1_score_class = 2 * (precision * recall) / (precision + recall) if (precision + recall) != 0 else 0

    print(f"\n📌 Class: {class_name}")
    print(f"🟩 True Positive (TP): {TP}")
    print(f"🟥 False Positive (FP): {FP}")
    print(f"🟨 False Negative (FN): {FN}")
    print(f"🟦 True Negative (TN): {TN}")
    print(f"\n📊 Class Metrics:")
    print(f"✅ Accuracy: {accuracy:.4f}")
    print(f"✅ Precision: {precision:.4f}")
    print(f"✅ Recall: {recall:.4f}")
    print(f"✅ F1-score: {f1_score_class:.4f}")

# Plot confusion matrix
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=test.class_indices.keys(),
            yticklabels=test.class_indices.keys())
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

# Visualize both original and Grad-CAM images (32 images total)
plt.figure(figsize=(20, 40))  # Tăng kích thước figure để chứa 32 ảnh

for i in range(16):
    idx = np.random.randint(0, len(X_test1))
    img_array1 = np.expand_dims(X_test1[idx], axis=0)
    img_array2 = np.expand_dims(X_test2[idx], axis=0)

    # Generate heatmap
    heatmap = make_gradcam_heatmap(img_array1, img_array2, model)

    # Load original image
    img = cv2.imread(test.filepaths[balanced_idx[idx]])
    img = cv2.resize(img, (224, 224))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Rescale heatmap
    heatmap = cv2.resize(heatmap, (img.shape[1], img.shape[0]))
    heatmap = np.uint8(255 * heatmap)
    heatmap_color = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)

    # Superimpose heatmap on image
    superimposed_img = cv2.addWeighted(img, 0.6, heatmap_color, 0.4, 0)

    # Get prediction info
    true_label = list(test.class_indices.keys())[y_test[idx]]
    pred_label = list(test.class_indices.keys())[y_pred[idx]]
    color = 'green' if y_test[idx] == y_pred[idx] else 'red'

    # Hiển thị ảnh gốc (hàng trên)
    plt.subplot(8, 4, 2*i+1)  # 8 hàng, 4 cột, vị trí lẻ
    plt.imshow(img)
    plt.title(f"Original\nTrue: {true_label}\nPred: {pred_label}", color=color)
    plt.axis('off')

    # Hiển thị ảnh Grad-CAM (hàng dưới)
    plt.subplot(8, 4, 2*i+2)  # 8 hàng, 4 cột, vị trí chẵn
    plt.imshow(superimposed_img)
    plt.title(f"Grad-CAM\nTrue: {true_label}\nPred: {pred_label}", color=color)
    plt.axis('off')

plt.tight_layout()
plt.show()



In [ ]:
from tensorflow.keras.preprocessing import image
# Đường dẫn tới ảnh
img_path = "real_vs_fake/IMG_0659.JPG"

# Nạp và tiền xử lý ảnh
img = image.load_img(img_path, target_size=(224, 224))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array /= 255.0  # Chuẩn hóa giá trị ảnh nếu cần (tùy thuộc vào mô hình của bạn)

# Dự đoán bằng mô hình
predictions = model.predict([img_array, img_array])  # Nếu mô hình là kết hợp MobileNetV2 và VGG16
predicted_class = np.argmax(predictions)

# Hiển thị kết quả dự đoán
label = "Real" if predicted_class == 1 else "Fake"
plt.imshow(img)
plt.title(f"Predicted: {label}")
plt.axis("off")
plt.show()

In [ ]:
from tensorflow.keras.preprocessing import image
# Đường dẫn tới ảnh
img_path = "real_vs_fake/00007.jpg"

# Nạp và tiền xử lý ảnh
img = image.load_img(img_path, target_size=(224, 224))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array /= 255.0  # Chuẩn hóa giá trị ảnh nếu cần (tùy thuộc vào mô hình của bạn)

# Dự đoán bằng mô hình
predictions = model.predict([img_array, img_array])  # Nếu mô hình là kết hợp MobileNetV2 và VGG16
predicted_class = np.argmax(predictions)

# Hiển thị kết quả dự đoán
label = "Real" if predicted_class == 1 else "Fake"
plt.imshow(img)
plt.title(f"Predicted: {label}")
plt.axis("off")
plt.show()

In [ ]:
from tensorflow.keras.preprocessing import image
# Đường dẫn tới ảnh
img_path = "real_vs_fake/0AW5ETVKV4.jpg"

# Nạp và tiền xử lý ảnh
img = image.load_img(img_path, target_size=(224, 224))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array /= 255.0  # Chuẩn hóa giá trị ảnh nếu cần (tùy thuộc vào mô hình của bạn)

# Dự đoán bằng mô hình
predictions = model.predict([img_array, img_array])  # Nếu mô hình là kết hợp MobileNetV2 và VGG16
predicted_class = np.argmax(predictions)

# Hiển thị kết quả dự đoán
label = "Real" if predicted_class == 1 else "Fake"
plt.imshow(img)
plt.title(f"Predicted: {label}")
plt.axis("off")
plt.show()

In [ ]:
# === PHẦN RETRAIN ===

# Sử dụng đặc trưng từ MobileNetV2 và VGG16 (đã được Global Average Pooling từ trước)
mobilenet_features = GlobalAveragePooling2D()(mobilenet.output)
vgg16_features = GlobalAveragePooling2D()(vgg16.output)

# Đồng bộ kích thước đặc trưng bằng Dense layers
mobilenet_features = Dense(512, activation='relu')(mobilenet_features)
vgg16_features = Dense(512, activation='relu')(vgg16_features)

# Kết hợp đặc trưng bằng Element-wise Addition
added_features = Add()([mobilenet_features, vgg16_features])

# Kết hợp đặc trưng bằng Element-wise Multiplication
multiplied_features = Multiply()([mobilenet_features, vgg16_features])

# Kết hợp cả hai phương pháp lại thành một vector tổng hợp
combined_retrain_features = Concatenate()([added_features, multiplied_features])

# Xây dựng các lớp fully connected cho retrain
x_retrain = Dense(512, activation='relu')(combined_retrain_features)
x_retrain = Dropout(0.3)(x_retrain)
output_retrain = Dense(2, activation='softmax')(x_retrain)

# Tạo mô hình retrain
model_retrain = Model(inputs=[mobilenet.input, vgg16.input], outputs=output_retrain)

# Compile mô hình retrain
model_retrain.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Hiển thị tóm tắt mô hình
model.summary()





In [ ]:
# Learning rate scheduler
def scheduler(epoch):
    if epoch <= 2:
        return 0.0001
    elif epoch > 2 and epoch <= 15:
        return 0.00001
    else:
        return 0.000001

lr_callbacks = tf.keras.callbacks.LearningRateScheduler(scheduler)

# Huấn luyện lại mô hình với tập dữ liệu mới (ví dụ: LCC-FASD hoặc tập dữ liệu hiện tại)
history_retrain = model_retrain.fit(
    train_dataset,  # Sử dụng tập train đã được chuẩn bị
    epochs=20,
    steps_per_epoch=len(train),
    validation_data=val_dataset,
    callbacks=[lr_callbacks],
    validation_steps=len(val)
)


In [ ]:
# === ĐÁNH GIÁ KẾT QUẢ ===

# So sánh độ chính xác giữa lần đầu huấn luyện và retrain
train_acc_first = hist.history['accuracy']  # Độ chính xác lần đầu huấn luyện
val_acc_first = hist.history['val_accuracy']
train_acc_retrain = history_retrain.history['accuracy']  # Độ chính xác retrain
val_acc_retrain = history_retrain.history['val_accuracy']

# Vẽ biểu đồ so sánh độ chính xác
plt.figure(figsize=(10, 5))
plt.plot(range(20), train_acc_first, label='Train Accuracy (First Train)')
plt.plot(range(20), val_acc_first, label='Validation Accuracy (First Train)')
plt.plot(range(20), train_acc_retrain, label='Train Accuracy (Retrain)')
plt.plot(range(20), val_acc_retrain, label='Validation Accuracy (Retrain)')
plt.title('Accuracy Comparison: First Train vs Retrain')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# So sánh mất mát giữa lần đầu huấn luyện và retrain
train_loss_first = hist.history['loss']
val_loss_first = hist.history['val_loss']
train_loss_retrain = history_retrain.history['loss']
val_loss_retrain = history_retrain.history['val_loss']

# Vẽ biểu đồ so sánh mất mát
plt.figure(figsize=(10, 5))
plt.plot(range(20), train_loss_first, label='Train Loss (First Train)')
plt.plot(range(20), val_loss_first, label='Validation Loss (First Train)')
plt.plot(range(20), train_loss_retrain, label='Train Loss (Retrain)')
plt.plot(range(20), val_loss_retrain, label='Validation Loss (Retrain)')
plt.title('Loss Comparison: First Train vs Retrain')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()



In [ ]:
# Lưu mô hình sau khi huấn luyện
model.save('real_vs_fake/checkpoints/model_multl_mobilenet_vgg16_v2.keras')


In [ ]:

model = load_model('real_vs_fake/checkpoints/model_multl_mobilenet_vgg16_v2.keras')

In [ ]:
# ================== DATA LOADING ==================
test_path = "real_vs_fake/real-vs-fake/test/"

# ImageDataGenerator for test set
test_datagen = ImageDataGenerator(rescale=1./255)

# Load test data
test = test_datagen.flow_from_directory(
    test_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='sparse',
    shuffle=False
)

print(f"Found {test.samples} images belonging to {len(test.class_indices)} classes.")

# Prepare test data for both inputs
X_test1, X_test2, y_test = [], [], []
for i in range(len(test)):
    X_batch, y_batch = test[i]
    X_test1.extend(X_batch)
    X_test2.extend(X_batch)
    y_test.extend(y_batch)

X_test1 = np.array(X_test1, dtype='float32')
X_test2 = np.array(X_test2, dtype='float32')
y_test = np.array(y_test, dtype='int')

# Balance the test set (500 from each class)
class_0_idx = np.where(y_test == 0)[0][:500]  # Fake
class_1_idx = np.where(y_test == 1)[0][:500]  # Real
balanced_idx = np.concatenate([class_0_idx, class_1_idx])

X_test1 = X_test1[balanced_idx]
X_test2 = X_test2[balanced_idx]
y_test = y_test[balanced_idx]

# ================== GRAD-CAM FUNCTION ==================
def make_gradcam_heatmap(img_array1, img_array2, model, last_conv_layer_name="out_relu"):
    # Create submodel that outputs both conv layer and predictions
    grad_model = Model(
        inputs=model.inputs,
        outputs=[model.get_layer(last_conv_layer_name).output, model.output]
    )

    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model([img_array1, img_array2])
        pred_index = tf.argmax(predictions[0])
        class_channel = predictions[:, pred_index]

    grads = tape.gradient(class_channel, conv_outputs)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
    conv_outputs = conv_outputs[0]
    heatmap = conv_outputs @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    return heatmap.numpy()

# ================== EVALUATION AND VISUALIZATION ==================
# Make predictions
predictions = model.predict([X_test1, X_test2])
y_pred = np.argmax(predictions, axis=1)

# Classification report
print("\n=== Classification Report ===")
print(classification_report(y_test, y_pred,
                          target_names=test.class_indices.keys(),
                          digits=4))

# Confusion matrix
plt.figure(figsize=(8, 6))
cm = confusion_matrix(y_test, y_pred)

# Calculate TP, TN, FP, FN for each class
# Tính các chỉ số tổng thể (nếu là bài toán binary classification)
if len(test.class_indices.keys()) == 2:  # Nếu là binary classification
    acc = accuracy_score(y_test, y_pred)  # Changed y_true to y_test
    prec = precision_score(y_test, y_pred)  # Changed y_true to y_test
    rec = recall_score(y_test, y_pred)  # Changed y_true to y_test
    f1 = f1_score(y_test, y_pred)  # Changed y_true to y_test
    tn, fp, fn, tp = cm.ravel()

    print(f"\n✅ Overall Accuracy: {acc:.4f}")
    print(f"✅ Overall Precision: {prec:.4f}")
    print(f"✅ Overall Recall: {rec:.4f}")
    print(f"✅ Overall F1-score: {f1:.4f}")

# In chi tiết cho từng class
print("\n=== 📊 Per-Class Statistics ===")
for i, class_name in enumerate(test.class_indices.keys()):
    TP = cm[i,i]
    FP = cm[:,i].sum() - TP
    FN = cm[i,:].sum() - TP
    TN = cm.sum() - TP - FP - FN

    # Tính các chỉ số cho từng class
    accuracy = (TP + TN) / (TP + TN + FP + FN) if (TP + TN + FP + FN) != 0 else 0
    precision = TP / (TP + FP) if (TP + FP) != 0 else 0
    recall = TP / (TP + FN) if (TP + FN) != 0 else 0
    f1_score_class = 2 * (precision * recall) / (precision + recall) if (precision + recall) != 0 else 0

    print(f"\n📌 Class: {class_name}")
    print(f"🟩 True Positive (TP): {TP}")
    print(f"🟥 False Positive (FP): {FP}")
    print(f"🟨 False Negative (FN): {FN}")
    print(f"🟦 True Negative (TN): {TN}")
    print(f"\n📊 Class Metrics:")
    print(f"✅ Accuracy: {accuracy:.4f}")
    print(f"✅ Precision: {precision:.4f}")
    print(f"✅ Recall: {recall:.4f}")
    print(f"✅ F1-score: {f1_score_class:.4f}")

# Plot confusion matrix
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=test.class_indices.keys(),
            yticklabels=test.class_indices.keys())
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

# Visualize both original and Grad-CAM images (32 images total)
plt.figure(figsize=(20, 40))  # Tăng kích thước figure để chứa 32 ảnh

for i in range(16):
    idx = np.random.randint(0, len(X_test1))
    img_array1 = np.expand_dims(X_test1[idx], axis=0)
    img_array2 = np.expand_dims(X_test2[idx], axis=0)

    # Generate heatmap
    heatmap = make_gradcam_heatmap(img_array1, img_array2, model)

    # Load original image
    img = cv2.imread(test.filepaths[balanced_idx[idx]])
    img = cv2.resize(img, (224, 224))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Rescale heatmap
    heatmap = cv2.resize(heatmap, (img.shape[1], img.shape[0]))
    heatmap = np.uint8(255 * heatmap)
    heatmap_color = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)

    # Superimpose heatmap on image
    superimposed_img = cv2.addWeighted(img, 0.6, heatmap_color, 0.4, 0)

    # Get prediction info
    true_label = list(test.class_indices.keys())[y_test[idx]]
    pred_label = list(test.class_indices.keys())[y_pred[idx]]
    color = 'green' if y_test[idx] == y_pred[idx] else 'red'

    # Hiển thị ảnh gốc (hàng trên)
    plt.subplot(8, 4, 2*i+1)  # 8 hàng, 4 cột, vị trí lẻ
    plt.imshow(img)
    plt.title(f"Original\nTrue: {true_label}\nPred: {pred_label}", color=color)
    plt.axis('off')

    # Hiển thị ảnh Grad-CAM (hàng dưới)
    plt.subplot(8, 4, 2*i+2)  # 8 hàng, 4 cột, vị trí chẵn
    plt.imshow(superimposed_img)
    plt.title(f"Grad-CAM\nTrue: {true_label}\nPred: {pred_label}", color=color)
    plt.axis('off')

plt.tight_layout()
plt.show()



In [ ]:
from tensorflow.keras.preprocessing import image
# Đường dẫn tới ảnh
img_path = "real_vs_fake/IMG_0659.JPG"

# Nạp và tiền xử lý ảnh
img = image.load_img(img_path, target_size=(224, 224))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array /= 255.0  # Chuẩn hóa giá trị ảnh nếu cần (tùy thuộc vào mô hình của bạn)

# Dự đoán bằng mô hình
predictions = model_retrain.predict([img_array, img_array])  # Nếu mô hình là kết hợp MobileNetV2 và VGG16
predicted_class = np.argmax(predictions)

# Hiển thị kết quả dự đoán
label = "Real" if predicted_class == 1 else "Fake"
plt.imshow(img)
plt.title(f"Predicted: {label}")
plt.axis("off")
plt.show()

In [ ]:

from tensorflow.keras.preprocessing import image
# Đường dẫn tới ảnh
img_path = "real_vs_fake/0AW5ETVKV4.jpg"

# Nạp và tiền xử lý ảnh
img = image.load_img(img_path, target_size=(224, 224))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array /= 255.0  # Chuẩn hóa giá trị ảnh nếu cần (tùy thuộc vào mô hình của bạn)

# Dự đoán bằng mô hình
predictions = model_retrain.predict([img_array, img_array])  # Nếu mô hình là kết hợp MobileNetV2 và VGG16
predicted_class = np.argmax(predictions)

# Hiển thị kết quả dự đoán
label = "Real" if predicted_class == 1 else "Fake"
plt.imshow(img)
plt.title(f"Predicted: {label}")
plt.axis("off")
plt.show()

In [ ]:
# ================== DATA LOADING ==================
test_path = "Dataset/Test/"

# ImageDataGenerator for test set
test_datagen = ImageDataGenerator(rescale=1./255)

# Load test data
test = test_datagen.flow_from_directory(
    test_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='sparse',
    shuffle=False
)

print(f"Found {test.samples} images belonging to {len(test.class_indices)} classes.")

# Prepare test data for both inputs
X_test1, X_test2, y_test = [], [], []
for i in range(len(test)):
    X_batch, y_batch = test[i]
    X_test1.extend(X_batch)
    X_test2.extend(X_batch)
    y_test.extend(y_batch)

X_test1 = np.array(X_test1, dtype='float32')
X_test2 = np.array(X_test2, dtype='float32')
y_test = np.array(y_test, dtype='int')

# Balance the test set (500 from each class)
class_0_idx = np.where(y_test == 0)[0][:500]  # Fake
class_1_idx = np.where(y_test == 1)[0][:500]  # Real
balanced_idx = np.concatenate([class_0_idx, class_1_idx])

X_test1 = X_test1[balanced_idx]
X_test2 = X_test2[balanced_idx]
y_test = y_test[balanced_idx]

# ================== EVALUATION AND VISUALIZATION ==================
# Make predictions
predictions = model.predict([X_test1, X_test2])
y_pred = np.argmax(predictions, axis=1)

# Classification report
print("\n=== Classification Report ===")
print(classification_report(y_test, y_pred,
                          target_names=test.class_indices.keys(),
                          digits=4))

# Confusion matrix
plt.figure(figsize=(8, 6))
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=test.class_indices.keys(),
            yticklabels=test.class_indices.keys())
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

# Visualize both original and Grad-CAM images (32 images total)
plt.figure(figsize=(20, 40))  # Tăng kích thước figure để chứa 32 ảnh

for i in range(16):
    idx = np.random.randint(0, len(X_test1))
    img_array1 = np.expand_dims(X_test1[idx], axis=0)
    img_array2 = np.expand_dims(X_test2[idx], axis=0)

    # Generate heatmap
    heatmap = make_gradcam_heatmap(img_array1, img_array2, model)

    # Load original image
    img = cv2.imread(test.filepaths[balanced_idx[idx]])
    img = cv2.resize(img, (224, 224))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Rescale heatmap
    heatmap = cv2.resize(heatmap, (img.shape[1], img.shape[0]))
    heatmap = np.uint8(255 * heatmap)
    heatmap_color = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)

    # Superimpose heatmap on image
    superimposed_img = cv2.addWeighted(img, 0.6, heatmap_color, 0.4, 0)

    # Get prediction info
    true_label = list(test.class_indices.keys())[y_test[idx]]
    pred_label = list(test.class_indices.keys())[y_pred[idx]]
    color = 'green' if y_test[idx] == y_pred[idx] else 'red'

    # Hiển thị ảnh gốc (hàng trên)
    plt.subplot(8, 4, 2*i+1)  # 8 hàng, 4 cột, vị trí lẻ
    plt.imshow(img)
    plt.title(f"Original\nTrue: {true_label}\nPred: {pred_label}", color=color)
    plt.axis('off')

    # Hiển thị ảnh Grad-CAM (hàng dưới)
    plt.subplot(8, 4, 2*i+2)  # 8 hàng, 4 cột, vị trí chẵn
    plt.imshow(superimposed_img)
    plt.title(f"Grad-CAM\nTrue: {true_label}\nPred: {pred_label}", color=color)
    plt.axis('off')

plt.tight_layout()
plt.show()

